# 🎥 Floutage Vidéo Intelligent d’Objets Sensibles

Ce notebook implémente exactement la **même logique de traitement vidéo**
que l’application **Streamlit** associée.

Fonctionnalités :
- Détection YOLOv8 (visages, plaques, écrans)
- Tracking persistant avec **KCF**
- Floutage ajustable
- Traitement frame par frame

⚠️ Ce notebook partage **les mêmes fonctions, variables et logique**
que l’application Streamlit (hors interface).


In [11]:
import cv2
from ultralytics import YOLO
import tempfile
import numpy as np
from tqdm import tqdm
from IPython.display import Video, display
import ipywidgets as widgets
import plotly.graph_objects as go


## 🔹 Chargement des modèles YOLO

Trois modèles sont utilisés :
- **Visages** : modèle YOLO spécialisé
- **Plaques** : modèle ALPR entraîné
- **Objets COCO** : modèle YOLOv8 standard


In [12]:
def charger_modeles():
    return {
        "face": YOLO("yolov8s-face-lindevs.pt"),
        "alpr": YOLO("best.pt"),
        "coco": YOLO("yolov8n.pt")
    }

models = charger_modeles()
OBJETS_COCO = ["laptop", "cell phone", "tv"]


## 🔹 Fonctions utilitaires

Ces fonctions permettent :
- De sécuriser les bounding boxes
- D’appliquer un floutage rectangulaire ou elliptique
- D’afficher un cercle de progression (facultatif)


In [13]:
def clamp_bbox(bbox, shape):
    x1, y1, x2, y2 = map(int, bbox)
    x1 = max(0, x1); y1 = max(0, y1)
    x2 = min(shape[1]-1, x2); y2 = min(shape[0]-1, y2)
    if x2 <= x1 or y2 <= y1:
        return None
    return x1, y1, x2, y2

def flouter_roi(frame, bbox, intensite, ellipse=False):
    x1, y1, x2, y2 = bbox
    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        return

    k = max(15, ((x2-x1)//3)|1) * intensite
    flou = cv2.GaussianBlur(roi, (k, k), 0)

    if ellipse:
        mask = np.zeros(roi.shape[:2], dtype=np.uint8)
        cx, cy = (x2-x1)//2, (y2-y1)//2
        rx = int((x2-x1) * 0.95 / 2)
        ry = int((y2-y1) * 0.95 / 2)
        cv2.ellipse(mask, (cx, cy), (rx, ry), 0, 0, 360, 255, -1)
        roi[mask == 255] = flou[mask == 255]
    else:
        roi[:] = flou


## 🔹 Paramètres globaux

- `FRAME_STEP` : fréquence de détection YOLO
- `MAX_MISSED` : nombre max de frames perdues avant suppression du tracker


In [14]:
FRAME_STEP = 1
MAX_MISSED = 1


## 🔹 Traitement principal de la vidéo

Cette fonction :
- Charge la vidéo uploadée
- Applique la détection + tracking
- Floute dynamiquement les objets
- Sauvegarde la vidéo finale


In [21]:
def traiter_video(video_bytes, use_face, use_alpr, use_coco, intensite_flou):
    temp_in = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    temp_in.write(video_bytes)
    temp_in.close()

    cap = cv2.VideoCapture(temp_in.name)
    w, h = int(cap.get(3)), int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Chemin exact que tu m'as donné
    out_path = "C:/Users/fatim/OneDrive/Bureau/Vidéos/video_floutée.mp4"
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    trackers = {"face": [], "alpr": [], "coco": []}
    last_bboxes = {"face": [], "alpr": [], "coco": []}
    missed = {"face": [], "alpr": [], "coco": []}
    frame_id = 0

    print("Début du traitement vidéo...")

    for _ in tqdm(range(total), desc="Traitement des frames"):
        ret, frame = cap.read()
        if not ret:
            break
        frame_id += 1

        # Détection YOLO
        if frame_id % FRAME_STEP == 0:
            small = cv2.resize(frame, (640, int(640*h/w)))
            for key, enabled, labels in [
                ("face", use_face, None),
                ("alpr", use_alpr, None),
                ("coco", use_coco, OBJETS_COCO)
            ]:
                if not enabled:
                    continue
                result = models[key](small, conf=0.4, verbose=False)[0]
                trackers[key], last_bboxes[key], missed[key] = [], [], []
                for box, cls in zip(result.boxes.xyxy, result.boxes.cls):
                    if labels and models[key].names[int(cls)] not in labels:
                        continue
                    bbox = [
                        box[0]*w/640, box[1]*h/small.shape[0],
                        box[2]*w/640, box[3]*h/small.shape[0]
                    ]
                    safe = clamp_bbox(bbox, frame.shape)
                    if safe:
                        flouter_roi(frame, safe, intensite_flou, ellipse=(key=="face"))
                        tracker = cv2.TrackerCSRT_create()
                        x1,y1,x2,y2 = safe
                        tracker.init(frame, (x1,y1,x2-x1,y2-y1))
                        trackers[key].append(tracker)
                        last_bboxes[key].append(safe)
                        missed[key].append(0)

        # Tracking persistant
        else:
            for key in trackers:
                new_trackers = []
                new_bboxes = []
                new_missed = []
                for i, tr in enumerate(trackers[key]):
                    ok, b = tr.update(frame)
                    if ok:
                        x, y, w0, h0 = map(int, b)
                        safe = clamp_bbox((x, y, x+w0, y+h0), frame.shape)
                        if safe:
                            flouter_roi(frame, safe, intensite_flou, ellipse=(key=="face"))
                            new_trackers.append(tr)
                            new_bboxes.append(safe)
                            new_missed.append(0)
                    else:
                        if i < len(last_bboxes[key]) and missed[key][i] < MAX_MISSED:
                            safe = last_bboxes[key][i]
                            flouter_roi(frame, safe, intensite_flou, ellipse=(key=="face"))
                            new_trackers.append(tr)
                            new_bboxes.append(safe)
                            new_missed.append(missed[key][i]+1)
                trackers[key] = new_trackers
                last_bboxes[key] = new_bboxes
                missed[key] = new_missed

        out.write(frame)

    cap.release()
    out.release()
    print(f"✅ Traitement terminé. Vidéo sauvegardée dans : {out_path}")
    return out_path


## 🔹 Interface interactive

L’utilisateur peut :
- Uploader une vidéo
- Choisir les types de floutage
- Régler l’intensité du flou
- Lancer le traitement en un clic


In [ ]:
upload_widget = widgets.FileUpload(accept=".mp4,.avi,.mov", multiple=False)
face_checkbox = widgets.Checkbox(value=True, description="Flouter les visages")
alpr_checkbox = widgets.Checkbox(value=True, description="Flouter les plaques")
coco_checkbox = widgets.Checkbox(value=True, description="Flouter les écrans")
intensite_slider = widgets.IntSlider(value=3, min=1, max=5, description="Intensité du flou")
process_button = widgets.Button(description="Lancer le floutage")
output_widget = widgets.Output()

display(upload_widget)
display(face_checkbox, alpr_checkbox, coco_checkbox, intensite_slider)
display(process_button, output_widget)

def get_uploaded_file(widget):
    if not widget.value:
        return None
    return widget.value[0]["content"] if isinstance(widget.value, tuple) \
        else list(widget.value.values())[0]["content"]

def on_button_click(b):
    with output_widget:
        output_widget.clear_output()

        video_bytes = get_uploaded_file(upload_widget)
        if video_bytes is None:
            print("⚠️ Veuillez uploader une vidéo.")
            return

        out_path = traiter_video(
            video_bytes,
            face_checkbox.value,
            alpr_checkbox.value,
            coco_checkbox.value,
            intensite_slider.value
        )


process_button.on_click(on_button_click)



FileUpload(value=(), accept='.mp4,.avi,.mov', description='Upload')

Checkbox(value=True, description='Flouter les visages')

Checkbox(value=True, description='Flouter les plaques')

Checkbox(value=True, description='Flouter les écrans')

IntSlider(value=3, description='Intensité du flou', max=5, min=1)

Button(description='Lancer le floutage', style=ButtonStyle())

Output()